In [1]:
from pyscf import gto,scf
import numpy as np
import ipyvolume as ipv
from numpy.linalg import norm as norm
from scipy.optimize import  minimize
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from pyscf.geomopt.berny_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109

In [1]:
import sys
sys.path.append("../Libs/")
%load_ext autoreload
%autoreload 2
from Morse import mpa
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM
from alch_deriv import alch_deriv
from FDcoeffs import *
%autoreload from AP_class import APDFT_perturbator as ap

In [3]:
nh3optg,oh2optg,fhoptg=np.load("optgeoms.npy")

In [4]:
l=3.3376
x=-(1/24)**.5*l
z=(2/3)**.5*l+x
ch4str="C 0 0 0; H 0 0  {}; H  0  {} {}; H {} {} {}; H {} {} {}".format(z,l/3**.5,x,l/2,-l*3**.5/6,x,-l/2,-l*3**.5/6,x)

In [13]:
CH4=gto.M(atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
CH4mf = scf.RHF(CH4)
CH4mf.scf()

converged SCF energy = -40.2151804034983


-40.215180403498316

In [6]:
H=gto.M(atom="H 0 0 0",unit="bohr",basis="pc-2",spin=1)
scf.RHF(H).scf()

converged SCF energy = -0.499878191497479


-0.4998781914974795

In [7]:
ch3rstr="C 0 0 0; H 0 0  2.043854241378284; H  0  1.9269642584472952 -0.6812847471260947; H 1.6688 -0.9634821292236474 -0.6812847471260947"

In [28]:
CH3r=gto.M(atom=ch3rstr,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])},spin=1)
CH3rmf = scf.RHF(CH3r)
CH3rmf.scf()

converged SCF energy = -39.5653567188866


-39.565356718886605

In [54]:
np.sum(CH4mf.mo_energy[:5])*2,CH4mf.energy_elec()[0]+CH4mf.energy_elec()[1]

(-27.57639999880104, -27.57640115699885)

In [25]:
e1=alch_deriv(CH4mf,[[4],[-1]])[2]

In [26]:
CH4.nao

101

In [119]:
CH4mf.mo_energy[:5],CH3mf.mo_energy[:5]

(array([-11.20183844,  -0.94571962,  -0.54688064,  -0.54688064,
         -0.54688064]),
 array([-1.08896294e+01, -6.02238522e-01, -2.45852258e-01, -2.45852258e-01,
        -5.95331173e-03]))

In [120]:
CH4mf.mo_energy[:5]+np.diag(e1)

array([-10.81074579,  -0.52290863,  -0.03524697,  -0.15408308,
        -0.13708015])

In [57]:
CH3=gto.M(atom=ch3rstr,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])},charge=-1)
CH3mf = scf.RHF(CH3)
CH3mf.scf()
CH3mf.mo_energy[:5]

converged SCF energy = -39.5136145548039


array([-1.08896294e+01, -6.02238522e-01, -2.45852258e-01, -2.45852258e-01,
       -5.95331173e-03])

In [128]:
CH3r.nelec

(5, 4)

In [62]:
CH4i=gto.M(atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])},charge=+1,spin=1)
CH4imf = scf.RHF(CH4i)
CH4imf.scf()

converged SCF energy = -39.7168298470357


-39.71682984703566

In [67]:
CH4mf.e_tot-CH4mf.mo_energy[4]   #Koopman's theorem

-39.66829975903329

In [118]:
CH3rmf.e_tot, CH3mf.e_tot,  CH3mf.e_tot-CH3mf.mo_energy[4]

(-39.565356718886605, -39.51361455480391, -39.50766124306982)

In [122]:
CH3mf.mo_energy[:6],CH3rmf.mo_energy[:6]

(array([-1.08896294e+01, -6.02238522e-01, -2.45852258e-01, -2.45852258e-01,
        -5.95331173e-03,  3.28296527e-01]),
 NPArrayWithTag([-11.22944763,  -0.91025232,  -0.55689092,  -0.55689092,
                  -0.13932217,   0.12338033]))

In [109]:
apc=ap(CH4mf,sites=[4])

In [111]:
apc.build_all()
apc.APDFT3(np.asarray([-1]))

-39.49614567169036

In [126]:
apc.APDFT3(np.asarray([-1]))-CH4mf.e_tot,apc.APDFT3(np.asarray([-1]))-CH4mf.e_tot-0.5

(0.7190347318079588, 0.21903473180795885)